# 뉴스 크롤링

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

In [6]:
# 카테고리 별 100개 기사 크롤링 (링크)
# 한 페이지에 20개

url_list = []   # url 저장 리스트
a_list = []     # a 태그 저장 리스트

for category in range(1):     # 6
    for page in range(1, 2):  # 1, 6
        url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        a_list.extend(soup.select(".type06_headline dt+dt a"))
        a_list.extend(soup.select(".type06 dt+dt a"))

        print("카테고리 : ", 100 + category, "페이지 : ", page)

for a in a_list:
    url_list.append(a["href"])  # 링크

카테고리 :  100 페이지 :  1


In [ ]:
url_list

In [12]:
# 수집한 링크의 제목, 본문, 날짜 크롤링

title = []
content = []
date = []

for url in url_list:

      browser.get(url)

      time.sleep(1)

      soup = BeautifulSoup(browser.page_source, "html.parser")

      title.extend(soup.select("#title_area span"))
      content.extend(soup.select("#dic_area"))
      date.extend(soup.select("._ARTICLE_DATE_TIME"))

In [25]:
# 위에서 수집한 태그 속에서 내용만 뽑아서 담는다.

title_list = []
content_list = []
date_list = []

for t in title:
  title_list.append(t.text)

for c in content:
  c = c.text.replace('\n', '')
  c = c.replace('\xa0', '')
  c = c.replace('\t', '')
  content_list.append(c)

for d in date:
  date_list.append(d["data-date-time"])

In [ ]:
# 데이터 프레임으로 변환

article_df = pd.DataFrame({"날짜" : date_list,
                           "제목" : title_list,
                           "본문" : content_list,
                           "링크" : url_list})

article_df

# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class

In [ ]:
pip install konlpy

In [38]:
# from konlpy.tag import Kkma
from konlpy.tag import Okt
from konlpy.utils import pprint
import re
import

In [65]:
article = """
'동료와 함께' 메시지로 글로벌 본격 공략…한미 음악방송 출연
걸그룹 르세라핌 (서울=연합뉴스) 이태수 기자 = 걸그룹 르세라핌이 27일 오후 1시 첫 영어 디지털 싱글 '퍼펙트 나이트'(Perfect Night)를 발표한다고 소속사 쏘스뮤직이 밝혔다.
'퍼펙트 나이트'는 '동료와 함께라면 완벽하지 않은 하루도 즐거울 수 있다'는 메시지를 담은 노래다.
르세라핌은 이날 KBS 2TV '뮤직뱅크'와 '더 시즌즈 - 악뮤의 오날오밤'을 시작으로 국내 음악방송에서 신곡 무대를 선보인다.
또 다음 달 2일(현지시간) 미국 그래미 뮤지엄 인기 온라인 퍼포먼스 시리즈 '글로벌 스핀 라이브'(Global Spin Live)에 출연하는 등 미국 활동에도 시동을 건다. 이 공연 티켓은 예매 시작 1분 만에 매진됐다.
르세라핌은 이번 신곡 뮤직비디오를 블리자드 엔터테인먼트의 유명 게임 '오버워치 2'와 협업해 제작했다. 이들은 다음 달 4일(현지시간) 미국 캘리포니아주 애너하임 컨벤션센터에서 열리는 게임 축제 '블리즈컨 2023'에도 출연해 공연을 펼친다."
"""

okt = Okt()

# for article in article_df["본문"]:
article = re.sub('\n','',article)
article = re.sub('\u200b','',article)
article = re.sub('\xa0','',article)
article = re.sub('([a-zA-Z])','',article)
article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',article)

article = okt.nouns(article)     # 명사
print(article)

['동료', '메시지', '글로벌', '본격', '공략', '한미', '음악', '방송', '출연', '걸그룹', '르', '세라핌', '서울', '연합뉴스', '이태수', '기자', '걸그룹', '르', '세라핌', '오후', '첫', '영어', '디지털', '싱글', '퍼펙트', '나이트', '를', '발표', '소속사', '스뮤', '직', '퍼펙트', '나이트', '동료', '하루', '수', '메시지', '노래', '르', '세라핌', '날', '뮤직뱅크', '더', '시즌즈', '악뮤', '오', '밤', '시작', '국내', '음악', '방송', '신곡', '무대', '또', '다음', '달', '미국', '그래미', '뮤', '지엄', '인기', '온라인', '퍼포먼스', '시리즈', '글로벌', '스핀', '라이브', '출연', '등', '미국', '활동', '시동', '건', '이', '공연', '티켓', '예매', '시작', '매진', '세라핌', '이번', '신곡', '뮤직비디오', '블리자드', '엔터테인먼트', '유명', '게임', '오버', '워치', '협업', '제작', '이', '다음', '달', '미국', '캘리포니아주', '애너하임', '컨벤션', '센터', '게임', '축제', '블리즈컨', '출연', '공연']
